In [10]:
import plotly.graph_objects as go

from time_series_data import choose_time_series_data

## Time series data Detail
- before running, you should edit n_dim parameter to 2 in yaml file.

In [11]:
data = choose_time_series_data("GRADUALLY_DENSE")

In [12]:
data.df.columns

Index(['col0', 'col1', 'step'], dtype='object')

In [13]:
data.sub_info.columns

Index(['color', 'text'], dtype='object')

In [14]:
n_steps = int(data.df.loc[:, "step"].max() + 1)

for t in range(n_steps):
    df = data.df.query(f"step == {t}")
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=df.loc[:, "col0"],
            y=df.loc[:, "col1"],
            mode="markers",
            marker={
                "color": data.sub_info.loc[:, "color"]
            }
        )
    )
    fig.show()

# Compare time series embedding to t-sne

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from sklearn.manifold import TSNE

from time_series_data import choose_time_series_data
from time_series_embedder import choose_time_series_embedder


def load_time_series_data(data_key="GRADUALLY_DENSE"):
    data = choose_time_series_data(data_key)
    print("== data.df == ")
    print(f"columns: {data.df.columns}")
    print(f"shape: {data.df.shape}")
    print("== data.subinfo == ")
    print(f"columns: {data.sub_info.columns}")
    print(f"shape: {data.sub_info.shape}")


def execute_t_sne(data_key="GRADUALLY_DENSE"):
    data = choose_time_series_data(data_key)
    print("== data.df == ")
    print(f"columns: {data.df.columns}")
    print(f"shape: {data.df.shape}")
    print("== data.subinfo == ")
    print(f"columns: {data.sub_info.columns}")
    print(f"shape: {data.sub_info.shape}")

    n_steps = int(data.df.loc[:, "step"].max()) + 1
    for t in range(n_steps):
        df = data.df.query(f"step == {t}")
        print(f"{t}/{n_steps}")
        coordinate = TSNE(n_components=2).fit_transform(df.filter(like="col", axis=1))
        rd_df = pd.DataFrame(coordinate, columns=[f"col{i}" for i in range(2)])

        fig = go.Figure()
        fig.add_trace(
            go.Scatter(
                x=rd_df.loc[:, "col0"],
                y=rd_df.loc[:, "col1"],
                mode="markers",
                marker={
                    "color": data.sub_info.loc[:, "color"],
                    "size": 8
                }
            )
        )
        fig.write_html(f"results/t_sne_{data_key}_{t}step.html")


def execute_time_series_embedder(data_key="GRADUALLY_DENSE", embedder_key="DYNAMIC_T_SNE", use_cache=True):
    data = choose_time_series_data(data_key)
    embedder = choose_time_series_embedder(embedder_key, data)
    embedder.embed(use_cache=use_cache, device='cuda')

    print("== embedder.coordinates == ")
    print(f"columns: {embedder.coordinates.columns}")
    print(f"shape: {embedder.coordinates.shape}")

    n_steps = int(embedder.coordinates.loc[:, "step"].max()) + 1
    for t in range(n_steps):
        print(f"{t}/{n_steps}")
        df = embedder.coordinates.query(f"step == {t}")
        fig = go.Figure()
        fig.add_trace(
            go.Scatter(
                x=df.loc[:, "col0"],
                y=df.loc[:, "col1"],
                mode="markers",
                marker={
                    "color": data.sub_info.loc[:, "color"],
                    "size": 8
                }
            )
        )
        fig.write_html(f"results/{embedder_key}_{data_key}_{t}step.html")

In [ ]:
execute_t_sne()
# execute_time_series_embedder()